# 0.0. Info Table

Monster Hunter 2 Dos Database App

    1. Lista de Quests separadas por Ranks.
    2. Lista de Monstros e suas caracteristicas.
    3. Lista de Kitchen Combination separadas por season.
    4. Lista de Itens.

In [2]:
import sqlite3
import requests
import pandas as pd

from bs4 import BeautifulSoup
from sqlalchemy import create_engine

# 1.0. List Itens.

## 1.1. List Itens Collect

In [135]:
path = '../data/page.html'

soup = BeautifulSoup( open(path).read(), 'html.parser' )

list_item = [list( filter( None, p.get_text().split('\n'))) for p in soup.find_all('tbody')[1].find_all('tr')[1:]]
list_item = pd.DataFrame( list_item )

## 1.2. List Itens Cleaning

In [136]:
# Drop Na & Rename Columns
list_item = list_item.dropna()
list_item.columns = ['id'] + [p.replace(' ', '_').lower() for p in list_item.iloc[0, :].tolist()][1:]
list_item = list_item[~list_item['item_name'].str.contains('Item')]

# # Individual Columns
list_item['quantity']  = list_item['quantity'].replace('two', '2')
list_item['quantity']  = list_item['quantity'].replace('(¡ç)', '99')
list_item['rare']      = list_item['rare'].replace('two', '2')
list_item['buy_price'] = [p+'z' if pd.notnull(p) else p for p in list_item['buy_price'].str.extract( '(\d+)' )[0].tolist()]
list_item['buy_price'] = list_item['buy_price'].fillna( 'No Sale' )
list_item['selling_price'] = [p+'z' if pd.notnull(p) else p for p in list_item['selling_price'].str.extract( '(\d+)' )[0].tolist()]
list_item.iloc[269, 1] = 'Sliced cactus'
list_item.iloc[:5, 1]  = ['Formulation I Introduction',
                         "Formulation II Beginner's edition",
                         'Formulation III Intermediate',
                         'Formulation IV Advanced Edition',
                         'Formulation V Master Edition']

# # Change Data Types
list_item['quantity'] = list_item['quantity'].astype('int64')
list_item['rare'] = list_item['rare'].astype('int64')

list_item = list_item.reset_index(drop=True)

# 2.0. List Items Verify

In [183]:
df = pd.read_excel( '../data/mh2dos_itens.xlsx' )

# Data Cleaning
df.columns = ['jp_name', 'name']+[p.replace(' ', '').lower() for p in df.iloc[2, :].tolist()][2:]
df.iloc[746, -2] = 'No Sale'
df = df.dropna()
df = df[~df['name'].str.contains('Coluna1')]

# # Individual Columns
df['carry']  = df['carry'].replace('(∞) ', 99)
df['rarity'] = df['rarity'].astype('int64')
df['buy']    = [p+'z' if pd.notnull(p) else p for p in df['buy'].str.extract( '(\d+)' )[0].tolist()]
df['buy']    = df['buy'].fillna('No Sale')
df['sell']   = [p+'z' if pd.notnull(p) else p for p in df['sell'].str.extract( '(\d+)' )[0].tolist()]

# # Chgange Dtypes
df['carry'] = df['carry'].astype('int64')
df['rarity'] = df['rarity'].astype('int64')

df = df.reset_index( drop=True )

## 2.1. Create New Dataframe

In [138]:
# Id, name, jp_name, carry, rarity, buy, sell

df1 = list_item[~list_item['item_name'].str.contains('(dummy)')]
df1 = df1[~df1['item_name'].str.contains('No bottle')]
df1 = df1[~df1['item_name'].str.contains('Bonus reward')]
df1 = df1[~df1['item_name'].str.contains('Wyvern')]
df1 = df1[~df1['item_name'].str.contains('Leftover food in the city')]
df1 = df1[~df1['item_name'].str.contains('Key of the dragon spear')]
df1 = df1[~df1['item_name'].str.contains('Sleeping ball')]
df1 = df1[~df1['item_name'].str.contains('Sedative ball')]
df1 = df1[~df1['item_name'].str.contains('Recovery ball')]
df1 = df1[~df1['item_name'].str.contains('Yakiniku ticket')]

df1 = df1.reset_index( drop=True )

C:\Users\T-Gamer\AppData\Local\Temp/ipykernel_12780/3191189436.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df1 = list_item[~list_item['item_name'].str.contains('(dummy)')]


## 2.2. Generate New DataFrame

In [215]:
df_raw = pd.concat( [df1[['id']], 
                     df[['name', 'jp_name', 'carry', 'rarity', 'buy', 'sell']]], 
                     axis=1 )

df_raw = df_raw.reset_index( drop=True )

## 2.3. List Itens Storange

In [214]:
list_item.to_csv('../data/itens.csv')

con = sqlite3.connect( '../sql_databases/itens.sqlite' )
c   = con.cursor()

query_create_table = '''
      CREATE TABLE item (
        id         TEXT,
        name       TEXT,
        jp_name    TEXT,
        carry      INTEGER,
        rarity     INTEGER,
        buy        TEXT,
        sell       TEXT ); '''

c.execute( query_create_table )
con.commit()

db  = create_engine( 'sqlite:///../sql_databases/itens.sqlite', echo=False )
con = db.connect()

df_raw.to_sql( 'item', con=con, if_exists='append', index=False )